## Import Packages

In [79]:
import pandas as pd
from dataclasses import dataclass
from datetime import datetime
import re
from dataclasses import field
from typing import Optional
from collections import defaultdict
import json
import pyarrow as pa
import pyarrow.parquet as pq

## Step 1: Load raw CSV and display first 3 rows

In [80]:
df = pd.read_csv("data/Sales Records.csv")
print("Table:", df.columns.tolist())
print(df.head(3))


Table: ['date', 'customer_id', 'product_id', 'price', 'quantity', 'coupon_code', 'shipping_city']
         date customer_id product_id    price  quantity coupon_code  \
0  2023-10-30    CUST0086       P001  1253.52         1  FREESHIP30   
1  2024-05-24    CUST0075       P020   253.25         2         NaN   
2  2023-10-06    CUST0079       P007    58.91         2         NaN   

  shipping_city  
0  Jacksonville  
1      New York  
2        Denver  


## Step 2: Pick the Right Container

Considering the three options to store each row: dictionaries, namedtuples, and classes.  
Classes are the way to go because they let us add logic like cleaning up data, transforming it, or calculating totals. Plus, they keep things neat and reusable for each transaction.


# structure and define Transaction class

In [81]:
@dataclass
class Transaction:
    date: str
    customer_id: str
    product_id: str
    price: float
    quantity: int
    coupon_code: str
    shipping_city: str
    discount_percent: float = field(default=0.0)
    purchase_age_days: Optional[int] = field(default=None)  # Derived field, set later

    def clean(self):
        try:
            self.price = float(self.price)
            if self.price < 0:
                self.price = 0.0
        except:
            self.price = 0.0
        self.coupon_code = self.coupon_code.upper() if isinstance(self.coupon_code, str) else "nan"

    def total(self):
        return self.price * self.quantity if self.price else 0


### Load Transaction data into the data class


In [82]:
def load_transactions(path: str) -> list[Transaction]:
    df = pd.read_csv(path)
    transactions = []
    for _, row in df.iterrows():        
        row_dict = row.to_dict()
        transaction = Transaction(**row_dict)    
        transactions.append(transaction)
    return transactions

# Step 5: Quick Profiling

In [83]:
def profile_transactions(transactions: list[Transaction]) -> None:
    valid_prices_list = [float(t.price) for t in transactions if isinstance(t.price, (int, float))]
    unique_cities = len(set(t.shipping_city for t in transactions))
    print("Quick Profiling: ")
    print(f"Min: {min(valid_prices_list)}")
    print(f"Mean: {sum(valid_prices_list)/len(valid_prices_list):.2f}")
    print(f"Max: {max(valid_prices_list)}")
    print(f"Unique Shipping Cities: {unique_cities}\n")


# Step 6: Injected 3 dirty values for demo.

In [84]:
def inject_dirty_val(transactions: list) -> None:
    if len(transactions) >= 3:
        transactions[0].price = "Nan"
        transactions[1].price = -150
        transactions[12].price = "aefada"

# Step 7: Cleaned transactions.

In [85]:
def clean_all_data(transactions: list[Transaction]) -> None:
    before = 0
    for t in transactions:
        if not isinstance(t.price, float):
            before += 1
    
    for t in transactions:
        t.clean()
    
    after = 0
    for t in transactions:
        if not isinstance(t.price, float):
            after += 1
    
    print(f"Cleaning results:")
    print(f"Before: {before} invalid prices")
    print(f"After: {after} invalid prices")

# Step 8: Calculating the discount percent based on the coupon_code.

In [86]:
def transform_transactions(transactions: list) -> None:
    for t in transactions:
        if t.coupon_code and isinstance(t.coupon_code, str):
            numbers = re.findall(r"\d+", t.coupon_code)
            if numbers:
                try:
                    t.discount_percent = int(numbers[0]) / 100
                except ValueError:
                    t.discount_percent = 0.0
            else:
                t.discount_percent = 0.0
        else:
            t.discount_percent = 0.0

### Calculating the 'days_since_purchase' to each transaction based on the purchased date

In [87]:
def feature_engineer(transactions: list) -> None:
    today = datetime.today()
    for t in transactions:
        t.purchase_age_days = (today - pd.to_datetime(t.date)).days


# Step 10: calculation to get Revenue per city.

In [88]:
def calculate_city_revenue(transactions: list) -> dict[str, float]:
    city_revenues = defaultdict(float)
    for transaction in transactions:        
        if isinstance(transaction.price, float):
            city_revenues[transaction.shipping_city] += transaction.total()
    return dict(city_revenues)


# Step 11: Saved cleaned data in JSON format and Parquet format

In [89]:
def save_transaction_data(transactions: list, json_path: str, parquet_path: str) -> None:
    cleaned_data = [t.__dict__ for t in transactions]
    with open(json_path, "w") as json_file:
        json.dump(cleaned_data, json_file)        
    df = pd.DataFrame(cleaned_data)
    table = pa.Table.from_pandas(df)
    pq.write_table(table, parquet_path)    
    print(f"Saved data to:\n- JSON: {json_path}\n- Parquet: {parquet_path}")

In [90]:
def main():
    tx = load_transactions("data/Sales Records.csv")
    profile_transactions(tx)
    inject_dirty_val(tx)
    clean_all_data(tx)
    transform_transactions(tx)
    feature_engineer(tx)
    ag = calculate_city_revenue(tx)
    print(F"City Revenue Summary:{ag}")
    save_transaction_data(tx, "data/transactions.json", "data/transactions.parquet")
    

# Code Starts Here (Main Function)

In [91]:
if __name__ == "__main__":
    main()

Quick Profiling: 
Min: 19.06
Mean: 268.40
Max: 1257.09
Unique Shipping Cities: 20

Cleaning results:
Before: 3 invalid prices
After: 0 invalid prices
City Revenue Summary:{'Jacksonville': nan, 'New York': 8989.23, 'Denver': 18430.690000000002, 'Columbus': 11435.619999999999, 'Houston': 13479.759999999997, 'Charlotte': 9965.210000000001, 'Austin': 8402.78, 'San Diego': 4654.06, 'Washington': 7611.620000000001, 'Dallas': 6764.02, 'Indianapolis': 5513.790000000001, 'San Jose': 11316.869999999999, 'San Antonio': 9362.599999999999, 'Los Angeles': 8810.460000000001, 'Boston': 15069.359999999997, 'Fort Worth': 8219.069999999998, 'Seattle': 12567.03, 'Phoenix': 11145.500000000002, 'Chicago': 13016.979999999996, 'Philadelphia': 7633.510000000001}
Saved data to:
- JSON: data/transactions.json
- Parquet: data/transactions.parquet


# Project Reflection: Benefits of Object-Oriented Programming

The adoption of Object-Oriented Programming (OOP) principles significantly enhanced the transaction processing system's architecture and functionality. By implementing a dedicated Transaction class, we achieved cohesive encapsulation of both data attributes (price, shipping city, etc.) and associated behaviors (data cleaning, validation, and calculations). This object-oriented approach offered three key advantages: First, it promoted code reusability, allowing transaction logic to be consistently applied throughout the system without duplication. Second, the self-contained nature of each transaction object simplified complex operations by localizing functionality - each object handles its own data processing while exposing a clean interface. Third, the modular design improved maintainability, as modifications could be made to transaction processing logic without cascading effects across the system. Particularly when handling hundreds of transaction records, this paradigm proved superior to procedural alternatives by reducing error rates through built-in validation and making the codebase more intuitive to debug and extend. The OOP structure ultimately delivered a more robust, organized, and scalable solution.

## Data Dictionary

Primary: Sales Records.csv
Secondary: product_detail.csv



| Field             | Type      | Description                                       | Source                |
|------------------|-----------|---------------------------------------------------|------------------------|
| date             | string    | Purchase date of the transaction                 | Sales Records.csv      |
| customer_id      | string    | Unique identifier for each customer              | Sales Records.csv      |
| product_id       | string    | Identifier for the product purchased             | Sales Records.csv      |
| product_name     | string    | Name of the product                              | product_detail.csv     |
| category         | string    | Category of the product                          | product_detail.csv     |
| description      | string    | Description of the product                       | product_detail.csv     |
| price            | float     | Unit price of the product                        | Sales Records.csv      |
| quantity         | int       | Quantity purchased                               | Sales Records.csv      |
| coupon_code      | string    | Promotional code used for the transaction        | Sales Records.csv      |
| shipping_city    | string    | City where the product is being shipped          | Sales Records.csv      |
| discount_pct     | float     | Numeric discount derived from coupon_code        | Derived                |
| days_since_purchase | int   | Days since the transaction date                  | Derived                |

